In [1]:
from pyspark.sql import SparkSession
import os
import configparser

In [2]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

In [3]:
%ls data

log-data.zip  song-data.zip


In [4]:
#!unzip data/song-data.zip

Archive:  data/song-data.zip
replace song_data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [5]:
%cat song_data/A/A/C/TRAACCG128F92E8A55.json

{"num_songs": 1, "artist_id": "AR5KOSW1187FB35FF4", "artist_latitude": 49.80388, "artist_longitude": 15.47491, "artist_location": "Dubai UAE", "artist_name": "Elena", "song_id": "SOZCTXZ12AB0182364", "title": "Setanta matins", "duration": 269.58322, "year": 0}

In [6]:
df = spark.read.json("song_data/A/A/C/TRAACCG128F92E8A55.json")

In [7]:
df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [8]:
df = spark.read.json("song_data/A/A/A")

In [9]:
%ls song_data/A/A/A/

TRAAAAW128F429D538.json  TRAAAFD128F92F423A.json  TRAAARJ128F9320760.json
TRAAABD128F429CF47.json  TRAAAMO128F1481E7F.json  TRAAAVG12903CFA543.json
TRAAADZ128F9348C2E.json  TRAAAMQ128F1460CD3.json  TRAAAVO128F93133D4.json
TRAAAEF128F4273421.json  TRAAAPK128E0786D96.json


In [19]:
df5 = spark.read.json("song_data/{A,B,C}/{A,B,C}/{A,B,C}")

In [20]:
df5.count()

71

In [25]:
df5.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [26]:
type(df5)

pyspark.sql.dataframe.DataFrame

In [34]:
df_artists = df5.select("artist_id", 
                        "artist_name",
                        "artist_location",
                        "artist_latitude",
                        "artist_longitude").\
        groupBy("artist_id", "artist_name", "artist_location", 
                "artist_latitude", "artist_longitude")


In [35]:
df_artists.count().show()

+------------------+--------------------+--------------------+---------------+----------------+-----+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|count|
+------------------+--------------------+--------------------+---------------+----------------+-----+
|ARPBNLO1187FB3D52F|            Tiny Tim|        New York, NY|       40.71455|       -74.00712|    1|
|ARBEBBY1187B9B43DB|           Tom Petty|     Gainesville, FL|           null|            null|    1|
|AR0IAWL1187B9A96D0|        Danilo Perez|              Panama|         8.4177|       -80.11278|    1|
|ARMBR4Y1187B9990EB|        David Martin|     California - SF|       37.77916|      -122.42005|    1|
|ARD0S291187B9B7BF5|             Rated R|                Ohio|           null|            null|    1|
|AR0RCMP1187FB3F427|    Billie Jo Spears|        Beaumont, TX|       30.08615|       -94.10158|    1|
|ARKRRTF1187B9984DA|    Sonora Santanera|                    |           null|    

In [36]:
df_artists = df5.select("artist_id", 
                        "artist_name",
                        "artist_location",
                        "artist_latitude",
                        "artist_longitude").\
        distinct()

In [37]:
df_artists.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_longitude: double (nullable = true)



In [38]:
#song_id, title, artist_id, year, duration
df_songs = df5.select("song_id", 
                      "title", 
                      "artist_id", 
                      "year", 
                      "duration").\
    distinct()

In [39]:
df_songs.show()

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOGOSOV12AF72A285E|   ¿Dónde va Chichi?|ARGUVEV1187B98BA17|1997|313.12934|
|SOTTDKS12AB018D69B|It Wont Be Christmas|ARMBR4Y1187B9990EB|   0|241.47546|
|SOBBUGU12A8C13E95D|Setting Fire to S...|ARMAC4T1187FB3FA4C|2004|207.77751|
|SOIAZJW12AB01853F1|          Pink World|AR8ZCNI1187B9A069B|1984|269.81832|
|SONYPOM12A8C13B2D7|I Think My Wife I...|ARDNS031187B9924F0|2005|186.48771|
|SOYMRWW12A6D4FAB14|The Moon And I (O...|ARKFYS91187B98E58F|   0| 267.7024|
|SOAOIBZ12AB01815BE|I Hold Your Hand ...|ARPBNLO1187FB3D52F|2000| 43.36281|
|SOBCOSW12A8C13D398|  Rumba De Barcelona|AR7SMBG1187B9B9066|   0|218.38322|
|SOWTBJW12AC468AC6E|Broken-Down Merry...|ARQGYP71187FB44566|   0|151.84934|
|SOQHXMF12AB0182363|     Young Boy Blues|ARGSJW91187B9B1D6B|   0|218.77506|
|SOGDBUF12A8